In [11]:
!pwd
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/marilynwaldman/doggievae/notebooks


In [12]:
#!pip install torch 

import torch
from torch import nn
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from src.mycategorical import CategoricalFeatures
from src.dataset import CustomDataset
from torch.utils.data import TensorDataset, DataLoader

In [13]:
# Set random seeds

torch.manual_seed(1)
torch.cuda.manual_seed(1)

In [14]:

# Read csv into dataframe and create custom data set

import pandas as pd
df = pd.read_csv("../data/ex4_2.csv")

# keep the length
df_len = len(df)

# Create a fake target column to "fake" dataset requirements.  This may not be needed.

df["target"] = -1

# Slice off the target column to make a numpy array of targets, then delete the column

targets_df =  df["target"]
del df['target']
full_data = df

# Create numpy arrays for data and targets, then create the CustomDataset

data = df.to_numpy()
targets = targets_df.to_numpy()








In [10]:
# Create numpy arrays for data and targets, then create the CustomDataset
data = df.to_numpy()
targets = targets_df.to_numpy()

model_dataset = CustomDataset(x = data, y = targets)
total_count = len(model_dataset)
print(total_count)
print(model_dataset[0]['x'].shape)


500
torch.Size([12])


In [15]:
print(model_dataset[0])

{'x': tensor([0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.]), 'y': tensor(-1)}


In [16]:
train_count = int(0.8 * total_count) 
valid_count = int(0.1 * total_count)
test_count = total_count - train_count - valid_count

In [17]:
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(model_dataset, (train_count, valid_count, test_count))

print(train_dataset[0])

{'x': tensor([1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1.]), 'y': tensor(-1)}


In [18]:
BATCH_SIZE = 256
NUM_WORKER = 1

#train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(model_dataset, (train_count, valid_count, test_count))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKER)  
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKER) 
test_loader  = torch.utils.data.DataLoader(test_dataset , batch_size=BATCH_SIZE, shuffle=False,num_workers=NUM_WORKER)
dataloaders = {'train': train_loader, 'val': valid_loader, 'test': test_loader}

In [16]:
# Create the data loader
for data in train_loader:
    print(data)
    break

{'x': tensor([[0., 0., 0.,  ..., 1., 1., 0.],
        [0., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 1.,  ..., 0., 0., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'y': tensor([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1

In [27]:
# Iterate over the dataloader

for data, target in test_loader:
    print(data)
    print(target)
    break


x
y


In [19]:
# Defining the device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
# Defining the model

d = 2


class VAE(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(12, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, d * 2)
        )

        self.decoder = nn.Sequential(
            nn.Linear(d, d ** 2),
            nn.ReLU(),
            nn.Linear(d ** 2, 12),
            nn.Sigmoid(),
        )

    def reparameterise(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x):
        #y = x.view(-1, 784)
        #print(y)
        mu_logvar = self.encoder(x.view(-1, 12)).view(-1, 2, d)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]
        z = self.reparameterise(mu, logvar)
        return self.decoder(z), mu, logvar

model = VAE().to(device)

In [21]:
# Setting the optimiser

learning_rate = 1e-3

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [22]:
# Reconstruction + KL divergence losses summed over all elements and batch

def loss_function(x_hat, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(
        x_hat, x.view(-1, 12), reduction='sum'
    )
    KLD = 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2))

    return BCE + KLD

In [29]:
# Training and testing the VAE

epochs = 10
codes = dict(μ=list(), logσ2=list(), y=list())
for epoch in range(0, epochs + 1):
    # Training
    if epoch > 0:  # test untrained net first
        model.train()
        train_loss = 0
        for data  in train_loader:
            x = data['x']
            y = data['y']

            x = x.to(device)
            # ===================forward=====================
            x_hat, mu, logvar = model(x)
            loss = loss_function(x_hat, x, mu, logvar)
            train_loss += loss.item()
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # ===================log========================
        print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')
    
    # Testing
    
    means, logvars, labels = list(), list(), list()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        for data in test_loader:
            x = data['x']
            y = data['y']

            x = x.to(device)
            # ===================forward=====================
            x_hat, mu, logvar = model(x)
            test_loss += loss_function(x_hat, x, mu, logvar).item()
            # =====================log=======================
            means.append(mu.detach())
            logvars.append(logvar.detach())
            labels.append(y.detach())
    # ===================log========================
    codes['μ'].append(torch.cat(means))
    codes['logσ2'].append(torch.cat(logvars))
    codes['y'].append(torch.cat(labels))
    test_loss /= len(test_loader.dataset)
    print(f'====> Test set loss: {test_loss:.4f}')
    #display_images(x, x_hat, 1, f'Epoch {epoch}')

====> Test set loss: 8.5625
====> Epoch: 1 Average loss: 8.6033
====> Test set loss: 8.5585
====> Epoch: 2 Average loss: 8.5434
====> Test set loss: 8.5545
====> Epoch: 3 Average loss: 8.5357
====> Test set loss: 8.5505
====> Epoch: 4 Average loss: 8.5755
====> Test set loss: 8.5466
====> Epoch: 5 Average loss: 8.5468
====> Test set loss: 8.5429
====> Epoch: 6 Average loss: 8.5309
====> Test set loss: 8.5393
====> Epoch: 7 Average loss: 8.5334
====> Test set loss: 8.5358
====> Epoch: 8 Average loss: 8.5522
====> Test set loss: 8.5326
====> Epoch: 9 Average loss: 8.5070
====> Test set loss: 8.5294
====> Epoch: 10 Average loss: 8.5177
====> Test set loss: 8.5263


In [31]:
# Generating a few samples

N = 16
z = torch.randn((N, d)).to(device)
sample = model.decoder(z)
#display_images(None, sample, N // 4, count=True)

In [ ]:
# Display last test batch

display_images(None, x, 4, count=True)

In [ ]:
# Choose starting and ending point for the interpolation -> shows original and reconstructed

A, B = 1, 14
sample = model.decoder(torch.stack((mu[A].data, mu[B].data), 0))
display_images(None, torch.stack(((
    x[A].data.view(-1),
    x[B].data.view(-1),
    sample.data[0],
    sample.data[1]
)), 0))

In [ ]:
# Perform an interpolation between input A and B, in N steps

N = 16
code = torch.Tensor(N, 20).to(device)
sample = torch.Tensor(N, 28, 28).to(device)
for i in range(N):
    code[i] = i / (N - 1) * mu[B].data + (1 - i / (N - 1) ) * mu[A].data
    # sample[i] = i / (N - 1) * x[B].data + (1 - i / (N - 1) ) * x[A].data
sample = model.decoder(code)
display_images(None, sample, N // 4, count=True)

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
from res.plot_lib import set_default

In [ ]:
set_default(figsize=(15, 4))

In [ ]:
X, Y, E = list(), list(), list()  # input, classes, embeddings
N = 1000  # samples per epoch
epochs = (0, 5, 10)
for epoch in epochs:
    X.append(codes['μ'][epoch][:N])
    E.append(TSNE(n_components=2).fit_transform(X[-1]))
    Y.append(codes['y'][epoch][:N])

In [ ]:
f, a = plt.subplots(ncols=3)
for i, e in enumerate(epochs):
    s = a[i].scatter(E[i][:,0], E[i][:,1], c=Y[i], cmap='tab10')
    a[i].grid(False)
    a[i].set_title(f'Epoch {e}')
    a[i].axis('equal')
f.colorbar(s, ax=a[:], ticks=np.arange(10), boundaries=np.arange(11) - .5)